In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pandas import Series

In [2]:
model = load_model('abd_model.h5')

W0718 09:38:48.382920 140398167582528 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0718 09:38:48.484642 140398167582528 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0718 09:38:48.485491 140398167582528 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init

In [3]:
import numpy as np
import pandas as pd
import pickle
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

with open('abd_variables.pkl', 'rb') as f:
    word_index, reverse_word_index = pickle.load(f)
    
def decode(ls):
    return ' '.join([reverse_word_index.get(i, '?') for i in ls])

def encode(text):
    text_ls = []
    for i in text.split():
        try:
            text_ls.append(word_index[i])
        except KeyError:
            print('Unhandled word \'' + i + '\'')
    return text_ls

def decode_single(text):
    return reverse_word_index.get(text)

def encode_single(text):
    return word_index.get(text)

def insert_start(ls):
    ls.insert(0, 1)
    return ls

In [4]:
def evaluate_log(log):
    test = StringIO(log)
    
    try:
        df_evaluate = pd.read_csv(test, sep=" ", header=None)
        df_evaluate[11], df_evaluate[12], df_evaluate[13] = df_evaluate[5].str.split(' ', 2).str
        df_evaluate.drop([0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 13], axis=1, inplace=True)
    except ValueError:
        return -1
    except KeyError:
        return -1
    
    if df_evaluate.at[0,11] != 'GET' and df_evaluate.at[0,11] != 'POST':
        return -1
    
    df_evaluate[12].replace({r'^/$': '<BASE_URL>'}, regex=True, inplace=True)
    df_evaluate[12].replace({r' \%[\%0-9A-Za-z]*': ' <PERCENT_URL> '},regex=True, inplace=True)
    df_evaluate[12].replace({r'\w+\d+\w+': ''},regex=True, inplace=True)
    df_evaluate[12].replace({r'\d{2,}': ''},regex=True, inplace=True)
    df_evaluate[12].replace({r'\[\d*\]': ' '},regex=True, inplace=True)
    df_evaluate[12].replace({'/': ' ', ':': ' ', '\.': ' ', '\?': ' ', '=': ' ', '\|': ' ', '&': ' '},regex=True, inplace=True)
    df_evaluate[11].replace({'GET': 2, 'POST': 3}, inplace=True)
    
    df_evaluate[12] = df_evaluate[12].str.lower()
    df_evaluate[12] = df_evaluate[12].apply(encode)
    df_evaluate[12] = df_evaluate[12].apply(insert_start)
    
    df_evaluate[6].replace({307: '<TEMPORARY_REDIRECT>', 400: '<BAD_REQUEST>', 404: '<NOT_FOUND>', 200: '<OK>', 301: '<MOVED_PERMANTLY>'}, inplace=True)
    df_evaluate[6] = df_evaluate[6].apply(encode_single)
    
    df_evaluate.columns = ['status', 'method', 'r_url']
    
    if len(df_evaluate.at[0, 'r_url']) < 11:
        for i in range (len(df_evaluate.at[0,'r_url']), 11):
            df_evaluate.at[0,'r_url'].append(0)
            
    for i in range(11):
        df_evaluate.at[0, 'r_url' + str(i)] = df_evaluate.at[0, 'r_url'][i]

    df_evaluate.drop(['r_url'], axis=1, inplace=True)
    
    single_evaluate = df_evaluate.values[0]
    
    print('Handled URL:')
    print(decode(single_evaluate))
    
    single_evaluate = (np.expand_dims(single_evaluate, 0))

    return round(model.predict(single_evaluate)[0][0], 4)

In [5]:
def evaluate_logs(logs_filename):
    max_lenght = 11
    
    df_evaluate = pd.read_csv(logs_filename, sep=" ", header=None)
    len_before = len(df_evaluate)
    
    df_evaluate[11], df_evaluate[12], df_evaluate[13] = df_evaluate[5].str.split(' ', 2).str
    
    df_evaluate.drop([0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 13], axis=1, inplace=True)
    
    df_evaluate[11].replace(r'^(?!GET$|POST$).*', pd.np.nan, inplace=True, regex=True)
    
    df_evaluate[12].replace({r'^/$': '<BASE_URL>'}, regex=True, inplace=True)
    df_evaluate[12].replace({r' \%[\%0-9A-Za-z]*': ' <PERCENT_URL> '},regex=True, inplace=True)
    df_evaluate[12].replace({r'\w+\d+\w+': ''},regex=True, inplace=True)
    df_evaluate[12].replace({r'\d{2,}': ''},regex=True, inplace=True)
    df_evaluate[12].replace({r'\[\d*\]': ' '},regex=True, inplace=True)
    df_evaluate[12].replace({'/': ' ', ':': ' ', '\.': ' ', '\?': ' ', '=': ' ', '\|': ' ', '&': ' '},regex=True, inplace=True)
    df_evaluate[11].replace({'GET': 2, 'POST': 3}, inplace=True)
    
    df_aux = df_evaluate[df_evaluate.isna().any(axis=1)]
    len_aux = len(df_aux)
    df_evaluate.dropna(inplace=True)
    
    df_evaluate[12] = df_evaluate[12].str.lower()
    df_evaluate[12] = df_evaluate[12].apply(encode)
    df_evaluate[12] = df_evaluate[12].apply(insert_start)
    
    df_evaluate[6].replace({307: '<TEMPORARY_REDIRECT>', 400: '<BAD_REQUEST>', 404: '<NOT_FOUND>', 200: '<OK>', 301: '<MOVED_PERMANTLY>'}, inplace=True)
    df_evaluate[6] = df_evaluate[6].apply(encode_single)
    
    df_evaluate.columns = ['status', 'method', 'r_url']
    aux_request_data = pad_sequences(df_evaluate['r_url'].values,
                                     value=word_index["<PAD>"],
                                     padding='post',
                                     maxlen=max_lenght)
    df_evaluate['r_url'] = Series(aux_request_data.tolist())
    
    
    df_aux = df_evaluate[df_evaluate.isna().any(axis=1)]
    len_aux += len(df_aux)
    df_evaluate.dropna(inplace=True)
    
    len_total = len(df_evaluate)
    
    for i in range(max_lenght):
        df_evaluate['r_url' + str(i)] = 0

    for i in df_evaluate.itertuples():
        for j in range(max_lenght):
            df_evaluate.at[i[0], 'r_url' + str(j)] = i[3][j]

    df_evaluate.drop(['r_url'], axis=1, inplace=True)

    to_evaluate_data = df_evaluate.values
    
    result = model.predict(to_evaluate_data)
    
    bad_logs = 0
    good_logs = 0
    for i in result:
        if round(i[0] * 100, 4) > 50:
            bad_logs += 1
        else:
            good_logs += 1
            
    print(len_before, len_total, len_aux)    
    print(bad_logs, good_logs)
    


In [14]:
log = '''
88.147.113.229 - - [06/Jul/2019:03:53:25 -0500] "GET /favicon.ico HTTP/1.1" 400 173 "-" "Mozilla/5.0" "-"
'''

evaluate_log(log)

Handled URL:
<BAD_REQUEST> <GET> <URL_START> favicon ico <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


0.9985

In [7]:
evaluate_logs('partial_log.log')

Unhandled word '%%%%'
Unhandled word '%%%%%%%%'
Unhandled word '%%6c%%6d%%6e%%5f%%%%%6e%%'
Unhandled word '%%%6d%%7a%6f%6e%%2f%%%6d%%7a%6f%6e%%2f%%%%6c%%'
Unhandled word '%%6a%%%5f%%6f%%6d'
Unhandled word '%5f%%%%%%%%5f%%6f%%6d%%'
Unhandled word '%%%%%%6c%5f%%6http'
Unhandled word '%%%%%2e%%%'
Unhandled word '%%%%%%%2e%%%'
Unhandled word '%2f%%6e%%%%2f%5c%%%%6e%6b%5c%%%%2f%%6e%%6f%6b%%%%6e%%%%6f%6e'
Unhandled word '%%%6c%6c%5f%%%%%5f%%%6e%%5f%%%%%'
Unhandled word '%6d%'
Unhandled word '%%%6c%6c%6f%%%%6e%6b%%%'
Unhandled word '%7b%%7b%%%%%6d%%%%%%%%%%4f%%%5f%%%%%%%%4f%%%%7d%7d'
Unhandled word '%7b%%7b%%%%%6d%%%%%%%%%%4f%%%5f%%%%%%%%4f%%%%7d%7d'
Unhandled word '%7b%%7b%%%%%6d%%%%%%%%%%4f%%%5f%%%%%%%%4f%%%%7d%7d'
Unhandled word '%7b%%7b%%%%%6d%%%%%%%%%%4f%%%5f%%%%%%%%4f%%%%7d%7d'
Unhandled word '%7b%%7b%%%%%6d%%%%%%%%%%4f%%%5f%%%%%%%%4f%%%%7d%7d'
Unhandled word '%2f%%6e%%%%2f%5c%%%%6e%6b%5c%%%%2f%%6e%%6f%6b%%%%6e%%%%6f%6e'
Unhandled word '%%%6c%6c%5f%%%%%5f%%%6e%%5f%%%%%'
Unhandled word '